<a href="https://colab.research.google.com/github/Nazhan99/Tensorflow_notes/blob/main/05_Transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transfer learning with TensorFlow part 2: Fine-Tuning

In [1]:
!nvidia-smi

Sun Feb 20 02:32:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Creating helper function

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-02-20 02:35:03--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-02-20 02:35:04 (44.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
#import helper function we are going to use in this notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data,  walk_through_dir

>if you are running this notebook in Google colab, when it times out colab will delete "helper_function".

## Get some data

use pretrained model using tf.keras.application and apply them to our own probelm (recognizing images of food).

In [6]:
# get 10% of training data of 10 fooc classes
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data("10_food_classes_10_percent.zip")

--2022-02-20 02:42:21--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 74.125.140.128, 108.177.15.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  95.8MB/s    in 1.7s    

2022-02-20 02:42:23 (95.8 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [8]:
#check out how many images and subdirectories are in our dataset
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/c

In [9]:
#create training and test directories paths
train_dir="10_food_classes_10_percent/train"
test_dir="10_food_classes_10_percent/test"


In [12]:
import tensorflow as tf

IMG_SIZE=(224,224)
BATCH_SIZE=32

train_data_10_percent= tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode="categorical",
                                                                           batch_size=BATCH_SIZE)

test_data_10_percent= tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                          image_size=IMG_SIZE,
                                                                          label_mode="categorical",
                                                                          batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [13]:
train_data_10_percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [15]:
#check out the class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [17]:
#see an example of a batch of data
for image, labels in train_data_10_percent.take(1):
  print(image,labels)

#need to normalize these tensors later

tf.Tensor(
[[[[1.96540817e+02 1.63826523e+02 1.25596939e+02]
   [2.00188782e+02 1.71040817e+02 1.36923462e+02]
   [1.79729584e+02 1.56158157e+02 1.29015305e+02]
   ...
   [1.92142792e+02 1.45581558e+02 4.90203133e+01]
   [1.86928558e+02 1.37571411e+02 4.26632690e+01]
   [1.89581711e+02 1.39867432e+02 4.72245750e+01]]

  [[1.93744888e+02 1.64887756e+02 1.28147949e+02]
   [1.94352036e+02 1.67270401e+02 1.36775513e+02]
   [1.86030624e+02 1.65244904e+02 1.38887756e+02]
   ...
   [2.03525391e+02 1.58984619e+02 6.37142258e+01]
   [1.89790787e+02 1.44714264e+02 5.16530380e+01]
   [1.87311157e+02 1.42831635e+02 5.14285355e+01]]

  [[1.89372437e+02 1.63948975e+02 1.33719391e+02]
   [1.86612244e+02 1.63397964e+02 1.35897964e+02]
   [1.82255096e+02 1.63448975e+02 1.41163269e+02]
   ...
   [2.09551117e+02 1.62403214e+02 6.81123962e+01]
   [2.02433594e+02 1.58091782e+02 6.29489326e+01]
   [1.86122177e+02 1.42907883e+02 4.86936035e+01]]

  ...

  [[2.26214264e+02 2.29214264e+02 2.36214264e+02]
   [2